KEYS: 相较于 v20240620 版本，增加了以下几点：

1. 移仓换月

In [79]:
import dai
import numpy as np
import pandas as pd
from datetime import datetime
from typing import Tuple
from bigmodule import M
from bigtrader.constant import OrderStatus, Direction, Offset

def initialize(context):
    # 特殊处理
    context.set_vmatch_at(0)        # 发单后不在当前tick成交

    # 策略参数
    context.short_grids = (0, 0)
    context.long_grids = (5000, 8000)
    context.grid_interval = context.data['grid_interval']
    context.order_qty = context.data['order_qty']
    context.close_interval = context.grid_interval

    # 策略变量
    context.previous_symbol = context.instruments[0]
    context.symbol = context.instruments[0]    # 交易合约
    context.curr_grid = 0           # 当前网格线
    context.next_grid = 0           # 下个网格线
    context.grid_info = {}          # 已开仓网格信息
    context.order_flag = 0          # 交易方向：1-做多；-1-做空
    context.stats_info_sum = {}     # 每日统计（全部）
    context.stats_info = []         # 每日统计：开仓次数和平仓次数
    context.isprint = 0             # 日志打印
    context.save_stats = True      # 存储CSV统计信息

    # 移仓换月
    context.rollover_trigger = False                        # 触发移仓换月的标识
    context.rollover_mapping = context.data['rollover']     # 日期和主力合约的映射
    context.rollover_info = {'close': {'id': -1, 'volume': 0, 'price': 0}, 'open': {'id': -1, 'volume': 0, 'price': 0}}       # 移仓换月下单相关信息
    context.rollover_flag, context.rollover_close, context.rollover_open = False, False, False      # 移仓换月标识


def before_trading_start(context, data):
    context.stats_info = [0, 0]     # 重置每日统计结果
    subsribe_symbols = list(set(list(context.get_account_positions().keys()) + [context.symbol]))
    subsribe_status = context.subscribe(subsribe_symbols)

    # 每日盘前初始化策略变量
    if not context.grid_info:
        # 第一次则判断昨日收盘价
        last_close = data.history(context.symbol, ['close'], 1, '1d')['close'].values[0]
        if (last_close >= context.short_grids[0]) and (last_close <= context.short_grids[1]):
            context.curr_grid = context.short_grids[0]
            context.next_grid = context.curr_grid + context.grid_interval
            context.order_flag = -1
        elif (last_close >= context.long_grids[0]) and (last_close <= context.long_grids[1]):
            context.curr_grid = context.long_grids[1]
            context.next_grid = context.curr_grid - context.grid_interval
            context.order_flag = 1
    else:
        context.grid_info = {k: v for k, v in context.grid_info.items() if v != 0}
        grid_info = context.grid_info.copy()
        context.grid_info = dict(sorted(grid_info.items()))
        min_grid = min(context.grid_info)
        max_grid = max(context.grid_info)
        if max_grid <= context.short_grids[1]:
            context.curr_grid = max_grid
            context.next_grid = context.curr_grid + context.grid_interval
            context.order_flag = -1
        elif min_grid >= context.long_grids[0]:
            context.curr_grid = min_grid
            context.next_grid = context.curr_grid - context.grid_interval
            context.order_flag = 1
    
    msg = '【盘前初始化 {}】时间: {}, 订阅合约: {}, 订阅返回: {}, 网格方向: {}, 当前网格: {}, 下个网格: {}, 已开仓网格: {}, 已有持仓: {}'.format(
        context.symbol, data.current_dt, subsribe_symbols, subsribe_status, context.order_flag, context.curr_grid,
        context.next_grid, context.grid_info, context.get_account_positions())
    context.write_log(msg, stdout=1)

def send_order(context, direction: str, offset: str, price: float, volume: int):
    price = int(price)
    send_status = None
    if (direction == Direction.LONG) and (offset == Offset.OPEN):          # 买开
        if volume != 0:
            send_status = context.buy_open(context.symbol, volume, price)
        context.curr_grid = price
        context.next_grid = context.curr_grid - context.grid_interval
        context.stats_info[0] += 1
    elif (direction == Direction.SHORT) and (offset == Offset.OPEN):       # 卖开
        if volume != 0:
            send_status = context.sell_open(context.symbol, volume, price)
        context.curr_grid = price
        context.next_grid = context.curr_grid + context.grid_interval
        context.stats_info[0] += 1
    elif (direction == Direction.LONG) and (offset == Offset.CLOSE or offset == Offset.CLOSETODAY):        # 买平
        if volume != 0:
            send_status = context.buy_close(context.symbol, volume, price)
        context.curr_grid = price
        context.next_grid = context.curr_grid + context.grid_interval
        context.stats_info[1] += 1
    elif (direction == Direction.SHORT) and (offset == Offset.CLOSE or offset == Offset.CLOSETODAY):       # 卖平
        if volume != 0:
            send_status = context.sell_close(context.symbol, volume, price)
        context.curr_grid = price
        context.next_grid = context.curr_grid - context.grid_interval
        context.stats_info[1] += 1
    msg = '【发单】下单回报: {}, 方向：{}, 开平: {}, 价格: {}, 数量: {}, 当前网格: {}, 下个网格: {}'.format(context.get_error_msg(send_status), direction, offset, price, volume, context.curr_grid, context.next_grid)
    context.write_log(msg, stdout=context.isprint)

    # 更新已开仓网格信息
    if offset == 1:
        context.grid_info[price] = 0

def rollover_close(context):
    """移仓换月平仓逻辑"""
    positions = context.get_account_positions()
    context.write_log(f'【移仓换月】当前持仓: {positions}', stdout=context.isprint)
    if not positions:
        return

    for symbol, info in positions.items():
        if symbol == context.symbol:
            continue
        long_qty = info.current_qty()[0]      # 做多持仓
        short_qty = info.current_qty()[1]     # 做空持仓
        last_price = info.last_price
        if long_qty >= 0:
            direction, price, volume = Direction.SHORT, int(last_price*0.90), long_qty
            send_status = context.sell_close(symbol, volume, price)
            msg = '【移仓换月-平仓】下单回报: {}, 方向：{}, 开平: {}, 价格: {}, 数量: {}'.format(context.get_error_msg(send_status), Direction.SHORT, Offset.CLOSE, price, volume)
        elif short_qty >= 0:
            direction, price, volume = Direction.LONG, int(last_price*1.1), short_qty
            send_status = context.buy_close(symbol, volume, price)
            msg = '【移仓换月-平仓】下单回报: {}, 方向：{}, 开平: {}, 价格: {}, 数量: {}'.format(context.get_error_msg(send_status), Direction.LONG, Offset.CLOSE, price, volume)
        else:
            msg, send_status, direction, price, volume = '【移仓换月】UNKNOWN', None, None, 0, 0
        context.rollover_info['close'] = {'id': context.get_last_order_key(), 'direction': direction, 'original_vol': volume, 'volume': 0, 'price': 0} if send_status == 0 else {'id': -1, 'direction': direction, 'original_vol': volume, 'volume': 0, 'price': 0}
        context.write_log(msg, stdout=context.isprint)

def rollover_open(context, curr_ask, curr_bid):
    """移仓换月开仓逻辑"""
    if context.rollover_info['close']['direction'] == Direction.LONG:
        volume, price = context.rollover_info['close']['original_vol'], int(curr_ask * 0.95)
        send_status = context.sell_open(context.symbol, volume, price)
        msg = '【移仓换月-开仓】下单回报: {}, 方向：{}, 开平: {}, 价格: {}, 数量: {}'.format(context.get_error_msg(send_status), Direction.SHORt, Offset.OPEN, price, volume)
    elif context.rollover_info['close']['direction'] == Direction.SHORT:
        volume, price = context.rollover_info['close']['original_vol'], int(curr_ask * 1.05)
        send_status = context.buy_open(context.symbol,  volume, price)
        msg = '【移仓换月-开仓】下单回报: {}, 方向：{}, 开平: {}, 价格: {}, 数量: {}'.format(context.get_error_msg(send_status), Direction.LONG, Offset.OPEN, price, volume)
    else:
        msg, send_status, price, volume = '【移仓换月】UNKNOWN', None, 0, 0
    context.rollover_info['open'] = {'id': context.get_last_order_key(), 'volume': 0, 'price': 0} if send_status == 0 else {'id': -1, 'volume': 0, 'price': 0}
    context.write_log(msg, stdout=context.isprint)

def handle_tick(context, tick):
    if ((tick.time_int > 205000000) & (tick.time_int < 205999999)) or ((tick.time_int > 85500000) & (tick.time_int < 85999999)):
        return

    # 移仓换月
    if context.rollover_flag is True:
        if context.rollover_close is True:      # 先平仓
            rollover_close(context)
            context.rollover_close = False
        if context.rollover_open is True:       # 再开仓
            rollover_open(context, tick.ask_price1, tick.bid_price1)
            context.rollover_open = False
        if (context.rollover_info['close']) and (context.rollover_info['open']) and (context.rollover_info['close']['volume'] == context.rollover_info['open']['volume']):
            # 根据差价更新网格
            price_diff = int(context.rollover_info['close']['price'] - context.rollover_info['open']['price'])
            new_grid_info = {}
            for k, v in context.grid_info.items():
                new_grid_info[k-price_diff] = v
            context.curr_grid -= price_diff
            context.next_grid -= price_diff
            context.write_log(f'【移仓换月-更新参数】当前网格: {context.curr_grid}, 下个网格: {context.next_grid}, 旧已开网格: {context.grid_info}, 新已开仓网格: {new_grid_info}', stdout=context.isprint)
            context.grid_info = new_grid_info
            context.rollover_flag = False
        return

    if tick.symbol != context.symbol:       # 移仓换月时会订阅两个合约的tick数据，网格交易的逻辑只针对主要的合约
        return

    # if tick.trading_day == '20230329':
    #     print(f"test: {tick.symbol}, {tick.datetime}, {tick.ask_price1}, {tick.bid_price1}, {context.curr_grid}, {context.next_grid}")

    # 做多逻辑
    if context.order_flag == 1:
        if tick.bid_price1 < context.next_grid:        # 做多开仓 - 买开
            if context.next_grid in context.grid_info:
                return
            msg = f'【触发买开】{tick.datetime}。卖一价: {tick.ask_price1}, 买一价: {tick.bid_price1}, 当前网格: {context.curr_grid}, 下个网格: {context.next_grid}'
            context.write_log(msg, stdout=context.isprint)
            send_order(context, direction=Direction.LONG, offset=Offset.OPEN, price=context.next_grid, volume=context.order_qty)
        elif tick.ask_price1 > context.curr_grid + context.close_interval:       # 做多平仓 - 卖平
            if context.curr_grid not in context.grid_info:
                return
            msg = f'【触发卖平】{tick.datetime}。卖一价: {tick.ask_price1}, 买一价: {tick.bid_price1}, 当前网格: {context.curr_grid}, 下个网格: {context.next_grid}'
            context.write_log(msg, stdout=context.isprint)
            send_order(
                context, direction=Direction.SHORT, offset=Offset.CLOSE,
                price=context.curr_grid + context.close_interval,
                volume=context.grid_info[context.curr_grid]
            )
    elif context.order_flag == -1:
        if tick.ask_price1 >= context.next_grid:       # 做空开仓 - 卖开
            if context.next_grid in context.grid_info:
                return
            msg = f'【触发卖开】{tick.datetime}。卖一价: {tick.ask_price1}, 买一价: {tick.bid_price1}, 当前网格: {context.curr_grid}, 下个网格: {context.next_grid}'
            context.write_log(msg, stdout=context.isprint)
            send_order(context, direction=Direction.SHORT, offset=Offset.OPEN, price=context.next_grid, volume=context.order_qty)
        elif tick.bid_price1 < context.curr_grid - context.close_interval:       # 做空平仓 - 买平
            if context.curr_grid not in context.grid_info:
                return
            msg = f'【触发买平】{tick.datetime}。卖一价: {tick.ask_price1}, 买一价: {tick.bid_price1}, 当前网格: {context.curr_grid}, 下个网格: {context.next_grid}'
            context.write_log(msg, stdout=context.isprint)
            send_order(
                context, direction=Direction.LONG, offset=Offset.CLOSE,
                price=context.curr_grid - context.close_interval,
                volume=context.grid_info[context.curr_grid]
            )

def handle_order(context, order):
    context.write_log('【委托回报】交易时间: {} {}, 标的: {}, 订单id: {}, 方向:{}, 开平: {}, 下单价格: {}, 下单数量: {}, 订单状态: {}, 成交均价: {}, 成交数量: {}'.format(
        order.insert_date, order.order_time, order.symbol, order.order_id, order.direction, order.offset, order.order_price,
        order.order_qty, order.order_status, order.avg_price, order.filled_qty
    ), stdout=context.isprint)

    # 移仓换月
    if (context.rollover_flag is True) and (context.rollover_info['close']):       # 平仓成交后开仓新合约
        if (order.order_key == context.rollover_info['close']['id']) and ((order.offset == Offset.CLOSE) or (order.offset == Offset.CLOSETODAY)):
            context.rollover_info['close']['volume'] = order.filled_qty
            context.rollover_info['close']['price'] = order.avg_price
            if (not context.rollover_info['open']) and (context.rollover_info['close']['original_vol'] == context.rollover_info['close']['volume']):        # 当平仓全部成交后，则开始开仓
                context.rollover_open = True
                return
    if (context.rollover_flag is True) and (context.rollover_info['open']):        # 记录新开仓的成交均价
        if (order.order_key == context.rollover_info['open']['id']) and (order.offset == Offset.OPEN):
            context.rollover_info['open']['volume'] = order.filled_qty
            context.rollover_info['open']['price'] = order.avg_price
            return

    # 撤单
    if order.order_status == OrderStatus.CANCELLED:
        if (order.offset == Offset.OPEN) and (context.grid_info[order.order_price] == 0):
            # 开仓网格还没成交则直接从已开仓网格信息中删除
            del context.grid_info[order.order_price]
        # 其他情况：开仓网格部分成交撤单 & 平仓撤单，则没有任何操作

    # 更新已开仓网格信息
    if order.offset == Offset.OPEN:
        context.grid_info[order.order_price] = order.filled_qty
    elif (order.offset == Offset.CLOSE) or (order.offset == Offset.CLOSETODAY):
        grid_price = order.order_price + context.close_interval if order.direction==Direction.LONG else order.order_price - context.close_interval
        if (grid_price in context.grid_info) and (order.filled_qty != 0):
            del context.grid_info[grid_price]

def handle_trade(context, trade):
    context.write_log('【成交回报】交易时间: {} {}, 标的: {}, 订单id: {}, 方向:{}, 开平: {}, 成交价格: {}, 成交数量: {}'.format(
        trade.trade_date, trade.trade_time, trade.symbol, trade.trade_id, trade.direction, trade.offset, trade.filled_price, trade.filled_qty
    ), stdout=context.isprint)

def after_trading(context, data):
    # 每日盘后确定下一日是否移仓换月
    curr_date = data.current_dt.strftime('%Y-%m-%d')
    if curr_date in context.rollover_mapping.keys():
        context.previous_symbol = context.symbol
        context.symbol = context.rollover_mapping[curr_date]
        context.rollover_info = {'close': {}, 'open': {}}
        context.rollover_flag, context.rollover_close, context.rollover_open = True, True, False
    else:
        context.rollover_info = {'close': {}, 'open': {}}
        context.rollover_flag, context.rollover_close, context.rollover_open = False, False, False

    today = data.current_dt.strftime('%Y-%m-%d')
    context.stats_info_sum[today] = [context.order_qty, context.grid_interval, context.stats_info]
    context.write_log(f'【盘后统计】{data.current_dt} 当日已开仓网格: {context.grid_info}', stdout=1)
    context.write_log(f'【盘后统计】{data.current_dt} 当日成交次数: {dict(sorted(context.stats_info_sum.items(), key=lambda item: item[0], reverse=True))}', stdout=1)

    # 保存统计数据
    df = pd.DataFrame(context.stats_info_sum).T.reset_index()
    df.columns = ['date', 'order_qty', 'interval', 'trade_nums']
    df[['open_nums', 'close_nums']] = pd.DataFrame(df['trade_nums'].tolist(), index=df.index)
    df.drop('trade_nums', axis=1, inplace=True)
    if context.save_stats is True:
        df.to_csv(f'stats_{context.grid_interval}.csv')

In [80]:
start_date = '2023-01-01 21:00:00'
end_date = '2024-06-25 15:00:00'

# 移仓换月
dom_df = dai.query("""
    SELECT date, instrument, dominant
    FROM cn_future_dominant
    WHERE instrument='SR8888.CZC'
""", filters={"date": [
    datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d'), 
    datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d')
]}).df().sort_values('date')
dom_df['previous'] = dom_df['dominant'].shift(1)
dom_df['rollover'] = 0
dom_df.loc[dom_df['dominant']!=dom_df['previous'], 'rollover'] = 1
dom_df.dropna(inplace=True)
dom_df['date'] = dom_df['date'].dt.strftime('%Y-%m-%d')
rollover = dom_df[dom_df['rollover']==1][['date', 'dominant']].set_index('date')['dominant'].to_dict()
symbols = dom_df['dominant'].unique().tolist()

# rollover['2023-01-05'] = 'SR305.CZC'
# del rollover['2023-01-31']
# rollover = dict(sorted(rollover.items()))

grid_interval = 10
order_qty = int(200/grid_interval)
data = {
    'instruments': symbols,
    'start_date': start_date,
    'end_date': end_date,
    'order_qty': order_qty,
    'grid_interval': grid_interval,
    'rollover': rollover,
}

M.bigtrader.v20(
    data=data,
    start_date='',
    end_date='',
    initialize=initialize,
    before_trading_start=before_trading_start,
    handle_tick=handle_tick,
    handle_trade=handle_trade,
    handle_order=handle_order,
    after_trading=after_trading,
    capital_base=100000000000,
    frequency='tick',
    product_type='期货',
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy='open',
    order_price_field_sell='open',
    benchmark='000300.SH',
    plot_charts=True,
    debug=True,
    backtest_only=False,
    m_cached=False
)

[2024-06-26 17:31:38] [info     ] bigtrader.v20 开始运行 ..
[2024-06-26 17:31:38] [info     ] 2023-01-01 21:00:00, 2024-06-25 15:00:00, instruments=7
[2024-06-26 17:31:38] [info     ] bigtrader module V2.0.3
[2024-06-26 17:31:38] [info     ] bigtrader engine v1.10.9 2024-06-17
[2024-06-26 17:31:38] [info     ] begin reading history data, 2023-01-01 21:00:00~2024-06-25 15:00:00, disable_cache:1
[2024-06-26 17:31:38] [info     ] reading benchmark data 000300.SH 2023-01-01 21:00:00~2024-06-25 15:00:00...
[2024-06-26 17:31:40] [info     ] reading daily data 2022-12-30 00:00:00~2024-06-25 15:00:00...
[2024-06-26 17:31:41] [info     ] cached_benchmark_ds:dai.DataSource("_4ae357933b444edeb254010a16b506e4")
[2024-06-26 17:31:41] [info     ] cached_daily_ds:None
2024-06-26 17:31:41.564746 init history datas... 
2024-06-26 17:31:41.567636 init history datas done. 
2024-06-26 17:31:41.573887 run_backtest() capital_base:100000000000, frequency:tick, product_type:future, date:20230103 ~ 2024-06-25 15:0